## Import Necessary Libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import sys

import model_metrics

print(model_metrics.__version__)

from sklearn.datasets import make_classification
from model_tuner import Model, loadObjects
import model_tuner

from eda_toolkit import ensure_directory
from model_metrics import (
    summarize_model_performance,
    show_calibration_curve,
    show_confusion_matrix,
    show_roc_curve,
    show_pr_curve,
    show_lift_chart,
    show_gain_chart,
)

0.0.2a


## Lasso Regression Metrics

In [2]:
model_path = os.path.join(os.pardir, "model_files/single_model_regression_results")
model_lasso = loadObjects(os.path.join(model_path, "lasso_regression_model.pkl"))

X_test = pd.read_parquet(os.path.join(model_path, "X_test.parquet"))
y_test = pd.read_parquet(os.path.join(model_path, "y_test.parquet"))

X_valid = pd.read_parquet(os.path.join(model_path, "X_valid.parquet"))
y_valid = pd.read_parquet(os.path.join(model_path, "y_valid.parquet"))

Object loaded!


## Summarize Model Performance

In [3]:
pipelines_or_models_lasso = [
    model_lasso,
]

# Model titles
model_titles = ["Lasso Regression", "Random Forest"]

In [4]:
summarize_model_performance(
    model=pipelines_or_models_lasso,
    X=X_test,
    y=y_test,
    model_type="regression",
    model_titles=model_titles,
    # model_threshold=thresholds,
    return_df=True,
    # overall_only=True,
    # decimal_places=2,
)

# print(model_summary_lasso)
# model_summary_lasso

,Model,Metric,Variable,Coefficient,P-value,MAE,MAPE,MSE,RMSE,Expl. Var.,R^2 Score
,Lasso Regression,Overall Metrics,,,,53.549,42.74,4106.445,64.082,0.316,0.29


In [5]:
import pandas as pd
from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.ensemble import RandomForestRegressor

# Load dataset
diabetes = load_diabetes(as_frame=True)["frame"]
X = diabetes.drop(columns=["target"])  # Cleaner way to get X
y = diabetes["target"]

# Split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Train Linear Regression (on unscaled data)
linear_model = LinearRegression()
linear_model.fit(X_train, y_train)

# Train Random Forest Regressor (on unscaled data)
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

# Scale features **only for Ridge**
scaler = StandardScaler()
X_train_scaled = pd.DataFrame(
    scaler.fit_transform(X_train), columns=X.columns, index=X_train.index
)
X_test_scaled = pd.DataFrame(
    scaler.transform(X_test), columns=X.columns, index=X_test.index
)

# Train Ridge Regression (on scaled data)
ridge_model = Ridge(alpha=1.0)
ridge_model.fit(X_train_scaled, y_train)

# Evaluate models using the correct input format
print("Linear Regression Score:", linear_model.score(X_test, y_test))  # Unscaled
print("Ridge Regression Score:", ridge_model.score(X_test_scaled, y_test))  # Scaled
print("Random Forest Score:", rf_model.score(X_test, y_test))  # Unscaled

Linear Regression Score: 0.4526027629719195
Ridge Regression Score: 0.45414652070698247
Random Forest Score: 0.4428225673999313


In [6]:
from model_metrics import summarize_model_performance

# Make a copy of X_test to avoid modifying the original
X_test_corrected = X_test.copy()

# Replace X_test with X_test_scaled **only for Ridge**
if ridge_model in [linear_model, ridge_model, rf_model]:
    X_test_corrected = X_test_scaled if ridge_model else X_test

# Run summarize_model_performance once
regression_metrics = summarize_model_performance(
    model=[linear_model, ridge_model, rf_model],
    model_titles=["Linear Regression", "Ridge Regression", "Random Forest"],
    X=X_test_corrected,  # Now Ridge gets the scaled version while others get unscaled
    y=y_test,
    model_type="regression",
    return_df=True,
)

# Display final table
regression_metrics

,Model,Metric,Variable,Coefficient,P-value,Feature Importance,MAE,MAPE,MSE,RMSE,Expl. Var.,R^2 Score
,Linear Regression,Overall Metrics,,,,,917.89,913.471,1234303.975,1110.992,-220.798,-231.969
,Linear Regression,Coefficient,age,37.904,,,,,,,,
,Linear Regression,Coefficient,sex,-241.964,,,,,,,,
,Linear Regression,Coefficient,bmi,542.429,,,,,,,,
,Linear Regression,Coefficient,bp,347.704,,,,,,,,
,Linear Regression,Coefficient,s1,-931.489,,,,,,,,
,Linear Regression,Coefficient,s2,518.062,,,,,,,,
,Linear Regression,Coefficient,s3,163.42,,,,,,,,
,Linear Regression,Coefficient,s4,275.318,,,,,,,,
,Linear Regression,Coefficient,s5,736.199,,,,,,,,
